In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import logging



In [5]:
mainUrl = "https://www.idealista.com"

listUrls = [("https://www.idealista.com/venta-viviendas/madrid/hortaleza/apostol-santiago/", "Hortaleza"),
            ("https://www.idealista.com/venta-viviendas/madrid/hortaleza/pinar-del-rey/", "Hortaleza"),
            ("https://www.idealista.com/venta-viviendas/madrid/hortaleza/canillas/", "Hortaleza"),
            ("https://www.idealista.com/venta-viviendas/madrid/hortaleza/conde-orgaz-piovera/", "Hortaleza"),
            ("https://www.idealista.com/venta-viviendas/madrid/hortaleza/palomas/", "Hortaleza"), 
            ("https://www.idealista.com/venta-viviendas/madrid/hortaleza/sanchinarro/", "Hortaleza"), 
            ("https://www.idealista.com/venta-viviendas/madrid/hortaleza/valdebebas-valdefuentes/", "Hortaleza"),
            ("https://www.idealista.com/venta-viviendas/madrid/hortaleza/virgen-del-cortijo-manoteras/", "Hortaleza"),
            ("https://www.idealista.com/venta-viviendas/madrid/ciudad-lineal/costillares/", "Ciudad Lineal")]

NEXT_PAGE_DELAY = 20
NEXT_ZONE_DELAY = 120

WEB_DRIVER_LOCATION = 'C:/Users/user/Downloads/chromedriver_win32/'

logging.basicConfig(filename='RealEstateInfo.log', level=logging.INFO, format='%(levelname)s-%(asctime)s: %(message)s')
logging.basicConfig(encoding='utf-8')
logging.basicConfig(filename='RealEstateDebug.log', encoding='utf-8', level=logging.DEBUG, format='%(levelname)s-%(asctime)s: %(message)s')
logging.basicConfig(encoding='utf-8')

logging.info('Opening new log session')

In [7]:
class DbPage:
    idList = []
    platformList = []
    titleList =  []
    priceList =  []
    parkingList = []
    habList = []
    nbanosList = []
    supList = []
    plantaList = []
    telefoneList =  []
    listHrefs = []
    listBarrio = []
    listZona = []
    listYear = []
    listMonth = []
    listDay = []
    listActive = []
    listYearInac = []
    listMonthInac = []
    listDayInac = []
    listCaract = []
    listComent = []
    
    
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.idList = []
        self.platformList = []
        self.titleList =  []
        self.priceList =  []
        self.parkingList = []
        self.habList = []
        self.nbanosList = []
        self.supList = []
        self.plantaList = []
        self.telefoneList =  []
        self.listHrefs = []
        self.listBarrio = []
        self.listZona = []
        self.listYear = []
        self.listMonth = []
        self.listDay = []
        self.listActive = []
        self.listYearInac = []
        self.listMonthInac = []
        self.listDayInac = []
        self.listCaract = []
        self.listComent = []

def setTitlePriceTelef(page, soup):
    page.titleList = soup.select("div.item-info-container > a.item-link")
    page.priceList = soup.select("div.item-info-container > div.price-row > span.item-price")
    
    for elem in soup.select("div.item-info-container > div.item-toolbar"):
        try:
            page.telefoneList.append( elem.select("span.icon-phone")[0].get_text() ) 
        except IndexError as error:
            page.telefoneList.append( "NA" )

def setIdealista(page, soup): #Encontrar id Idealista del piso
    date =  datetime.now()
    for elem in soup.select("div.item-info-container > a.item-link"):
        page.idList.append( int(elem["href"].split("/")[2]) )
        page.platformList.append("idealista")
        page.listYear.append( date.year )
        page.listMonth.append( date.month )
        page.listDay.append( date.day )
        
def setHabSupPl(page, soup):  #Encontrar Habitaciones, superficie y planta
    isHab = lambda a : True if "hab" in a else False
    isSup = lambda a : True if "m²" in a else False
    isPlanta = lambda a : True if ("planta".lower() in a.lower() or "Bajo".lower() in a.lower() or "exterior".lower() in a.lower() ) else False
    set_bit = lambda value, bit : value | (1<<bit)  #zero index

    for elem in soup.select("div.item-info-container"):
        auxStr = ""
        bitmap = 0  # hab,sup,planta as bits 000, 101, etc
        for subelem in elem.select("span.item-detail"):
            auxStr = subelem.get_text()
            if isHab(auxStr):
                page.habList.append( auxStr )
                bitmap = set_bit(bitmap, 2)
            if isSup(auxStr):
                page.supList.append( auxStr )
                bitmap = set_bit(bitmap, 1)
            if isPlanta(auxStr):
                page.plantaList.append( auxStr )
                bitmap = set_bit(bitmap, 0)
        if (bitmap & 0x1) == False:
            page.plantaList.append( "NA" )
        elif (bitmap & 0x2)>>1 == False:
            page.supList.append( "NA" )
        elif (bitmap & 0x4)>>2 == False:
            page.habList.append( "NA" )

def setParking(page, soup):  # Encontrar datos Garaje
    for elem in soup.select("div.item-info-container > div.price-row"):
        auxStr = ""
        parkingElem = elem.select("span.item-parking")
        if( not parkingElem):
            auxStr += "No Garaje "
        else:
            auxStr += parkingElem[0].get_text()
        page.parkingList.append( auxStr )

def setUrl(page, soup):  #Encontrar URL
    for elem in soup.select("div.item-info-container > a.item-link"):
        page.listHrefs.append( "https://www.idealista.com" + elem['href'] )
        
def insertPageInDf(page, df):
    print("debug6: page.listZona[-1]: ", page.listZona[-1])
    for i in range(0,len(page.titleList)-1):
        row = [page.idList[i],
               page.platformList[i],
               page.titleList[i].get_text(),
               page.priceList[i].get_text(),
               page.habList[i],
               'NA',
               page.supList[i],
               page.plantaList[i],
               page.parkingList[i],
               page.telefoneList[i],
               page.listHrefs[i],
               page.listBarrio[i],
               page.listZona[i],
               page.listYear[i],
               page.listMonth[i],
               page.listDay[i],
               'Yes',
               'NA',
               'NA',
               'NA',
               'NA',
               'NA'
               ]
        if (df.IdPlat == int(page.idList[i])).any() :  #Evaluar tambien en caso de que si el precio es distinto, actualizar
            continue
        df.loc[len(df)] = row

def getNextPageUrl(soup, baseUrl): #compute next page URL
    out = ""
    try:
        out = mainUrl + soup.select("div.pagination > ul > li.next > a")[0]["href"]
    except IndexError as error:
        print("Last page in entry")
        out = "Last page in entry"
    return out

def setBarrioZona(Barrio, url, soup, page):
    Zona = url.split('/')[-2]
    for elem in soup.select("div.item-info-container > a.item-link"):
        page.listBarrio.append( Barrio )
        page.listZona.append( url.split('/')[-2].capitalize() )

def OpenBrowserAndFetchData(url):
    #Open webpage with selenium webdriver, extract page source, and close webdriver in order to avoid scaring target website 
    #with the multiple repeated requests that webdrive does.
    options = webdriver.ChromeOptions()
    options.page_load_strategy = 'eager'
    options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")
    ChromeDriverFull = (WEB_DRIVER_LOCATION + 'chromedriver.exe')
    driver = webdriver.Chrome(r"{}".format(ChromeDriverFull), options=options)
    driver.get(url)
    content = driver.page_source
    driver.quit()

    #Get Beautiful Soup Html object to work with
    soup = BeautifulSoup(content, 'html.parser')    
    return soup


def getDf():
    df = pd.DataFrame(columns=['IdPlat','Plat','Titulo','Precio','Hab','N_Banos','Sup','Planta','Garaje','Telefono','Url','Barrio','Zona','Year','Month', 'Day', 'Active', 'YearInact','MonthInact', 'DayInact', 'Caract', 'Comentario'])
    df = df.astype({"IdPlat": int})
    return df

def WorkZona(df, listUrls_entry, mainUrl):
    maxLoopCounts = 20
    counter = 0

    url = listUrls_entry[0]
    Barrio = listUrls_entry[1]
    
    while url != 'Last page in entry':
        if counter >= maxLoopCounts:
            break

        page = DbPage()

        logging.info("Working.. url: " + url + ", counter: " + str(counter) )
        print("Working.. url: ", url, ", counter: ", counter)
        soup = OpenBrowserAndFetchData(url)

        setTitlePriceTelef(page, soup)
        setIdealista(page, soup)
        print(page.idList)
        setHabSupPl(page, soup)
        setParking(page, soup)
        setUrl(page, soup)
        setBarrioZona(Barrio, url, soup, page)
        insertPageInDf(page, df)

        #logging.info("Next Url: " + url)
        print(df[['IdPlat', 'Titulo']])
        url = getNextPageUrl(soup, mainUrl)
        
        logging.info("Next Url: " + url)
        print("Next Url: ", url)
        if url == 'Last page in entry':
            break
        time.sleep(NEXT_PAGE_DELAY) # NEXT_PAGE_DELAY seconds delay after request
        counter += 1

In [8]:
#load pandas db
df_Db_in = getDf()
try:
    df_Db_in = pd.read_csv('dbPisos.csv', encoding='cp1252', index_col=False, keep_default_na=False)
except FileNotFoundError:
    print("First do one round of botting, to generate initial dbPisos.csv")

df_Db_in = df_Db_in.astype({"IdPlat": int})

df_Db_in

First do one round of botting, to generate initial dbPisos.csv


,IdPlat,Plat,Titulo,Precio,Hab,N_Banos,Sup,Planta,Garaje,Telefono,...,Zona,Year,Month,Day,Active,YearInact,MonthInact,DayInact,Caract,Comentario


Main Bot Script

In [ ]:
currentDateTime = lambda: datetime.now().strftime("%Y/%m/%d/ %H:%M:%S")

print("Starting job: ", currentDateTime())
logging.info( "Starting job: " + currentDateTime() )

df_today = getDf()

for elem in listUrls:
    print( currentDateTime(), ", Current Url: ", elem[0] )
    logging.info("Current Url: " + elem[0] )
    WorkZona(df_today, elem, mainUrl)
    
    print("Sleeping 2 min to be nice to servers: ", currentDateTime() )
    logging.info("Sleeping 2 min to be nice to servers: " + str(currentDateTime())  )
    time.sleep(NEXT_ZONE_DELAY)


Starting job:  2021/01/07/ 22:27:23
2021/01/07/ 22:27:23 , Current Url:  https://www.idealista.com/venta-viviendas/madrid/hortaleza/apostol-santiago/
Working.. url:  https://www.idealista.com/venta-viviendas/madrid/hortaleza/apostol-santiago/ , counter:  0
[92089313, 92108754, 91949551, 90218172, 87295228, 88574743, 88448484, 84992553, 90128541, 92268519, 92125508, 88827177, 92219308, 90870221, 90987895, 88881308, 91959762, 91094571, 91148458, 92006700, 91752074, 88759681, 82298148, 27004286, 1833289, 92017339, 82917307, 1645789, 84834215, 89902044]
debug6: page.listZona[-1]:  Apostol-santiago
      IdPlat                                             Titulo
0   92089313  Piso en calle del Golfo de Salonica, Apóstol S...
1   92108754  Piso en calle de San Pedro de Cardeña, Apóstol...
2   91949551                   Piso en Apóstol Santiago, Madrid
3   90218172                   Piso en Apóstol Santiago, Madrid
4   87295228  Piso en calle de Añastro, Apóstol Santiago, Ma...
5   88574743  C

In [ ]:
df_today


In [ ]:
df_Db_out = pd.concat([df_Db_in,df_today]).drop_duplicates(subset = ['IdPlat'], keep='first')
df_Db_out

In [ ]:
#Write pandas df to db
df_Db_out.to_csv('dbPisos.csv', encoding='cp1252',index=False)

Experimental

In [ ]:

def FindNewAndInact(df_Db_in, df_today):
    #Find new and inactive elements (none duplicates)
    return pd.concat([df_Db_in,df_today]).drop_duplicates(subset = ['IdPlat'], keep=False)

def FindNewDf(df_Db_in, df_today, df_Db_out):
    #Find new
    df_newAndInact = FindNewAndInact(df_Db_in, df_today)
    newPos = pd.concat([df_today,df_newAndInact]).duplicated(subset = ['IdPlat'], keep='first')
    newPos =newPos[~newPos.index.duplicated(keep='last')]
    return df_Db_out.loc[newPos] #List them

def FindInactiveDf(df_Db_in, df_today, df_Db_out):
    #Find inactive
    df_newAndInact = FindNewAndInact(df_Db_in, df_today)
    inactPos = pd.concat([df_Db_in,df_newAndInact]).duplicated(subset = ['IdPlat'], keep='last')
    inactPos =inactPos[~inactPos.index.duplicated(keep='first')]
    return df_Db_out.loc[inactPos]  #Out of those, set them inactive

def MarkInactiveDf(df_Db_in, df_today, df_Db_out):
    date =  datetime.now()
    df_newAndInact = FindNewAndInact(df_Db_in, df_today)
    inactPos = pd.concat([df_Db_in,df_newAndInact]).duplicated(subset = ['IdPlat'], keep='last')
    inactPos =inactPos[~inactPos.index.duplicated(keep='first')]
    df_Db_out.loc[inactPos, 'Active'] = 'No'
    df_Db_out.loc[inactPos, 'YearInact'] = date.year
    df_Db_out.loc[inactPos, 'MonthInact'] = date.month
    df_Db_out.loc[inactPos, 'DayInact'] = date.day

    
df_newAndInact = FindNewAndInact(df_Db_in, df_today)
df_newAndInact
    

In [ ]:
df_new = FindNewDf(df_Db_in, df_today, df_Db_out)
df_new

In [ ]:
df_Inactive = FindInactiveDf(df_Db_in, df_today, df_Db_out)
df_Inactive
MarkInactiveDf(df_Db_in, df_today, df_Db_out)
df_Inactive

In [ ]:
#Write pandas df to db
df_Db_out.to_csv('dbPisos2.csv', encoding='cp1252',index=False)